# PathFinding Data PreProcessing
* find subject data
* find experiment and eyetracking files
* combine experiment files for single subject to consolidate
* combine eyetracking files for single subject and pre-process
* performance analysis of PathFinding experiments

## Configuration

In [1]:
# General configuration
import os

# data_directory: str
#     Path to a directory to store data.
data_directory = '.'

# install_missing_packages: bool
#     A flag indicating if missing packages should be automatically installed
install_missing_packages = True

# use_conda: bool
#     A flag indicating if conda should be used for software installation.
#     If False, pip will be used. The default is to use conda if jupyter
#     is run in a conda environment.
use_conda = 'CONDA_EXE' in os.environ

## Checking for missing packages

In [2]:
import importlib

def check_package(package, pip_pkg: str = None, conda_pkg: str = None):
    """Check if a given package is installed. If missing install
    it (if global flag `install_missing_packages` is True) either with
    pip or with conda (depending on `use_conda`).
    """
    if importlib.util.find_spec(package) is not None:
        return  # ok, package is already installed

    if not install_missing_packages:
        raise RuntimeError(f"{package} is not installed!")

    if use_conda:
        import conda.cli
        conda.cli.main('conda', 'install',  '-y', conda_pkg or package)
    else:
        import subprocess
        import sys            
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pip_pkg or package])
        
# This is to exit cells without error tracebacks (cosmetic purpose)
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

## Creating the required environment (skip if already done)

Running the following cell will create a file graphs.yml that can be used to setup a conda environment containing the required packages. If you already downloaded the file from my GitHub, skip the next cell and create the env directly from it.

In [3]:
%%writefile graphs.yml
name: graphs
channels:
  - conda-forge
  - defaults
dependencies:
  - python=3.6
  - jupyter
  - imageio
  - imageio-ffmpeg
  - matplotlib
  - scikit-image
  - opencv
  - networkx
  - pandas
  - statsmodels

Writing graphs.yml


### Environment Creation
To create the environment, open the terminal, go to the directory where you stored the graphs.yml file (the directory of the notebook) and type
conda env create -f graphs.yml
After running this command you have to activate the environment (Linux/MacOS: conda activate graphs, Windows: activate graphs) and then reopen the notebook in that environment.

## Main Part

### Imports and directory information

In [15]:
import os
import json
import numpy as np
import re
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nX
import glob
import scipy.cluster.vq as clusters
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from pandas.plotting import autocorrelation_plot as AC_plot 
#from statsmodels.graphics import tsaplots
#from statsmodels.tsa.stattools import acf
from skimage.filters import gaussian
from mpl_toolkits.mplot3d import Axes3D 
import time
import random



In [124]:
OG_DATA_PATH = './'

condition = 'Single' # Single, Dyadic, SingleC
DATA_PATH = './Data {}/'.format(condition)

PROCESSED_DATA_PATH = './Results/' + condition + '/'

# Extracting all subject IDs from the data folder
# Getting the Folder without hidden files in ascending order 
DATA_FOLDER = sorted([f for f in os.listdir(DATA_PATH) if not f.startswith('.')], key=str.lower)

subIDs = []
for sub in DATA_FOLDER:
    if sub[0].isdigit():
        subIDs.append(int(sub[0:4]))
    else:
        pass
subIDs = np.unique(subIDs)


#subIDs = [1023] # remove to do for all subIDs
print(subIDs)

[1004 1005 1008 1010 1011 1013 1017 1018 1019 1021 1022 1023 1054 1055
 1056 1057 1058 1068 1069 1072 1073 1074 1075 1077 1079 1080]


# Combining Pathfinding Experiment Files per Subject

In [24]:
EXPT_FILE_ATTRS = ['ParticipantID', 'PathsReversed', 'IsDyadic', 'IsLeader'];


def map_dict(a_dictionary):
    a_subset = {key: value for key, value in a_dictionary.items() if key in EYE_FILE_ATTRS};
    return a_subset;

# For each subject in subject ID folder, combine all experiment files
# and save as a unified final experiment file in Result folder
for sub in subIDs:
    # Get all files for subject
    SUB_FILES = list(filter(lambda x: str(sub)+"_" in x, DATA_FOLDER));
    # Sort files into experiment files and eyetracking files
    EYETRACKING_FILES = list(filter(lambda x: 'EyeTracking' in x, SUB_FILES));
    EYETRACKING_FILES.sort();
    
    EXPT_FILES = list(filter(lambda x: condition + 'PathFinding' in x, SUB_FILES));
    EXPT_FILES.sort();
    
    EXPT_DATA = {};
        
    # For each experiment file
    for expt_file in EXPT_FILES:
        
        # Read JSON data
        with open(DATA_PATH + expt_file) as f:
            try:
                subject_session = json.loads(f.read())
            except:
                print("\tJSON file " + expt_file + " is not valid!")
                continue;
        
        # If there are more than one experiment files, combine the data of the paths if the 
        # other trial and subject information matches
        if len(EXPT_DATA) > 0:
            print([subject_session[x] for x in EXPT_FILE_ATTRS]);
            print([EXPT_DATA[x] for x in EXPT_FILE_ATTRS]);
            if(not all([subject_session[x] == EXPT_DATA[x] for x in EXPT_FILE_ATTRS])):
                raise Exception("Experiment File Data " + expt_file + " Does Not Match!")
            else:
                EXPT_DATA['Concatenated'] = True;
                EXPT_DATA['TrialData'] += subject_session['TrialData'];
        
        else:
            EXPT_DATA = subject_session;
            EXPT_DATA['Concatenated'] = False;
        
    # Check if correct number of paths present in experiment file
    if len(EXPT_DATA['TrialData']) != 10:
        raise Exception("Incorrent number of paths in file for Subject " + str(sub));
        break;
        
    # Save the final combined file in results to be accessed by eyetracking collection script
    expt_file_name = PROCESSED_DATA_PATH + str(sub) + "_"+condition+"Pathfinding_Final.json";
    with open(expt_file_name, 'w') as fp:
        json.dump(EXPT_DATA, fp);
    EXPT_DATA = {};
    

In [20]:
# Store Time limits for each path condition
path_time_limits_A = [300, 305, 234, 332, 268, 394, 152, 383, 332];
path_time_limits_B = path_time_limits_A.copy();
path_time_limits_B.reverse();


# Leader : follower map
dyadic_pair_map = {
    '1021' : '1022',
    '1023' : '1013',
    '1005' : '1055',
    '1074' : '1069',
    '1008' : '1058',
    '1054' : '1004',
    '1011' : '1017',
    '1018' : '1057'
}

# Generate random session IDs to group leader and follower from same session easily in analysis
dyadic_session_ids = {}
for leader in dyadic_pair_map.keys():
    _id = random.randint(1000,9999);
    dyadic_session_ids[leader] = _id;
    dyadic_session_ids[dyadic_pair_map[leader]] = _id;
    
print(dyadic_session_ids);

{'1021': 7274, '1022': 7274, '1023': 5062, '1013': 5062, '1005': 5361, '1055': 5361, '1074': 6989, '1069': 6989, '1008': 4957, '1058': 4957, '1054': 5575, '1004': 5575, '1011': 1300, '1017': 1300, '1018': 5128, '1057': 5128}


### Combine the eyetracking data
* Loop through all subjects
* extract the session data
* combine the data
* save it

In [71]:
# --------- Preparation ---------

Session_save_bool = False # set to True if you want to save each individual session as csv
Exploration_save_bool = True # set to True if you want to save the complete exploration as csv
subcount = 0

# column name list for dataframe
col_names =  ['PathNumber',
          'timeStampDataPointStart',
          'timeStampDataPointEnd',
          'serverTimeStampDataPointStart',
          'serverTimeStampDataPointEnd',
          'hitObjectColliderName', 
          'ordinalOfHit',
          'BitMask',
          'hitPointOnObject.x',
          'hitPointOnObject.y',
          'hitPointOnObject.z',
          'hitObjectColliderBoundsCenter.x',
          'hitObjectColliderBoundsCenter.y',
          'hitObjectColliderBoundsCenter.z',
            'hmdPosition.x',
             'hmdPosition.y',
             'hmdPosition.z']


# TODO: Add server times to nohitdict
NoHit_dict = {'hitPointOnObject': {'x': 'NaN',
                                   'y': 'NaN',
                                   'z': 'NaN'},
              'hitObjectColliderName': 'NoHit',
              'hitObjectColliderBoundsCenter': {'x': 'NaN',
                                                'y': 'NaN',
                                                'z': 'NaN'},
              'ordinalOfHit': 'NaN',
              'PathNumber': 'NaN',
              'timeStampDataPointStart': 'NaN',
              'timeStampDataPointEnd': 'NaN',
              'serverTimeStampDataPointStart' : 'NaN',
              'serverTimeStampDataPointEnd' : 'NaN',
              'BitMask': 'NaN',
              'hmdPosition': {'x': 'NaN',
                              'y': 'NaN',
                              'z': 'Nan'}
              }

# Remove machine time stamps if single
if 'Single' in condition:
    col_names.pop(3);
    col_names.pop(3);
    NoHit_dict.pop('serverTimeStampDataPointStart');
    NoHit_dict.pop('serverTimeStampDataPointEnd');



# --------------------------- MAIN PART ---------------------------


# --------- first layer - subject loop ---------

for subject in subIDs:
    subcount +=1
    print('Subject ' + str(subject) + ' started - ' + str(subcount) + '/' + str(len(subIDs)))
    
    # Create empty dataframe for later concatenation
    complete_exploration_df = pd.DataFrame(columns = col_names)
    
    # get the data files according to the subject
    subject_folder = sorted([f for f in DATA_FOLDER 
                             if f.startswith(str(subject)+'_EyeTrackingData_')], 
                            key=str.lower) 
    
    # the following works as long as the data name format is as follows:
    # 'subjectID'_Expl_S_'SessionNumber'_ET_'EyeTrackingSessionNumber'_'UnixTimestamp'.json
    folder_files = list()
       
    # loop through the subject folder and save all numbers
    for file in subject_folder:
        folder_files.append(re.findall(r'\d+', file))
    
    if len(folder_files) == 0:
        raise Exception("No eyetracking files in " + condition +" condition for subject " + str(subject))
    # Extract all SubIDs (only one), (and Timestamps)
    SubID, UnixTimestamp1, _ = map(list, zip(*folder_files))
    
    # Get the experiment JSON file for the subject
    sub_expt_data = {}
    sub_expt_file_name = PROCESSED_DATA_PATH + str(subject) + "_"+condition+"Pathfinding_Final.json";
    try:
        with open(sub_expt_file_name, 'r') as fp:
            sub_expt_data = json.load(fp);
    except:
        raise Exception("Could not read experiment file for subject " + str(subject));
    
# --------- second layer - exploration session loop ---------
    s = 0;
    complete_hitpoints_df = pd.DataFrame(columns = col_names)
    previous_path_num = -1.0;
    # loop over separate eye tracking sessions
    for ET_session in subject_folder:
        s+=1
        #print("\tSession " + str(s) + " started")

            # open the JSON file as dictionary
        with open(DATA_PATH + ET_session) as f:
            try:
                subject_session = json.loads(f.read())
            except:
                print("\tJSON file " + ET_session + " is not valid!")

        hitpoint_list = list() # create hitpoint list
        
        # check if trial is valid:
        trial_valid = subject_session['trials'][0]['trialIsValid']
        if not trial_valid:
            raise Exception("Excluding " + ET_Session + " because trial is not valid!");
            continue;
        
        sub_id = subject_session['trials'][0]['participantId']
        if str(sub_id) != str(subject):
            raise Exception("Excluding " + ET_Session + " because subject IDs don't match!");
            continue;
        
        # start timestamp of the session 
        start_time = subject_session['trials'][0]['timeTrialMeasurementStarted']
        
        # get trial id (path number)
        path_num = subject_session['trials'][0]['trialId']
        
        expt_path_start_time = sub_expt_data['TrialData'][path_num]['StartMachineTimeStamp'];
        expt_path_end_time = sub_expt_data['TrialData'][path_num]['EndMachineTimeStamp'];
        
        machine_time_stamp_diff = 0.0;
        if 'Single' not in condition and path_num != previous_path_num:
            expt_path_start_srvtime_mS = sub_expt_data['TrialData'][path_num]['StartServerTimeStamp'];
            machine_time_stamp_diff = expt_path_start_time - expt_path_start_srvtime_mS / 1000;
            previous_path_num = path_num;

        # amount of datapoints 
        Len_subses = len(subject_session['trials'][0]['dataPoints'])

        # for loop appending each data point rayCastHit Data
        # afterwards adding the timestamp to the dict 
        # passing if there is (1) no raycast hit and (2) if there is only one raycast hit
        for each in subject_session['trials'][0]['dataPoints']:

            # account for noHits 
            if each['rayCastHitsCombinedEyes'] == []:
                hitpoint_list.append(NoHit_dict)
            else:
                # append data point
                hitpoint_list.append(each['rayCastHitsCombinedEyes'][0]) 
                # add path number, timestamp and bitmask
                idx = len(hitpoint_list)-1
                hitpoint_list[idx]['PathNumber'] = path_num
                
                # Exclude the data point if it lies outside of the times for a path
                if each['timeStampDataPointStart'] < expt_path_start_time or each['timeStampDataPointStart'] > expt_path_end_time:
                    hitpoint_list[idx]['PathNumber'] += 0; # Change this to some other high number to exclude


                # Add the server time stamp if dyadic
                if 'Single' not in condition:
                    hitpoint_list[idx]['serverTimeStampDataPointStart'] = (each['timeStampDataPointStart'] - machine_time_stamp_diff) * 1000.0;
                    hitpoint_list[idx]['serverTimeStampDataPointEnd'] = (each['timeStampDataPointEnd'] - machine_time_stamp_diff) * 1000.0;
                hitpoint_list[idx]['timeStampDataPointStart'] = each['timeStampDataPointStart'] - start_time
                hitpoint_list[idx]['timeStampDataPointEnd'] = each['timeStampDataPointEnd'] - start_time
                hitpoint_list[idx]['BitMask'] = each['combinedGazeValidityBitmask']
                
                # Add HMD position
                hitpoint_list[idx]['hmdPosition'] = each['hmdPosition'];


            try: 
                # append data point of second raycast hit if it exists
                hitpoint_list.append(each['rayCastHitsCombinedEyes'][1])
                # add Path number, timestamp and bitmask
                idx = len(hitpoint_list)-1
                hitpoint_list[idx]['PathNumber'] = hitpoint_list[idx-1]['PathNumber'];
                
                # Add the server time stamp if dyadic
                if 'Single' not in condition:
                    hitpoint_list[idx]['serverTimeStampDataPointStart'] = (each['timeStampDataPointStart'] - machine_time_stamp_diff) * 1000.0;
                    hitpoint_list[idx]['serverTimeStampDataPointEnd'] = (each['timeStampDataPointEnd'] - machine_time_stamp_diff) * 1000.0;
                
                hitpoint_list[idx]['timeStampDataPointStart'] = each['timeStampDataPointStart'] - start_time
                hitpoint_list[idx]['timeStampDataPointEnd'] = each['timeStampDataPointEnd'] - start_time
                hitpoint_list[idx]['BitMask'] = each['combinedGazeValidityBitmask']
                
                

            except:
                pass

        # normalize the hitpoint dictionary to get dataframe
        hitpoints_df = pd.json_normalize(hitpoint_list)

        print("\tET: " + str(s) + " normalized")

        complete_hitpoints_df = complete_hitpoints_df.append(hitpoints_df)


    # --------- Saving each Session ---------

    # If you want to save each session separately, set 'Session_save_bool' to True
    if Session_save_bool == True:
        try:
            if len(subject_data) > 0:
                complete_hitpoints_df.to_csv(PROCESSED_DATA_PATH
                                             + str(subject)
                                             + "_CompleteSession"
                                             + "_" + condition
                                             + "_Hitpoints.csv")
                print("\t"
                      + str(subject)
                      + " session "
                      + "_" + condition
                      + " saved ")
            else: 
                print("\t"
                      + str(subject)
                      + " - Session "
                      + "_" + condition
                      + " is empty!")
        except:
            print("\tCould not save subject "
                  + str(subject)
                  + " session "
                  + "_" + condition
                  + "!")



    # fill the complete exploration dataframe with the separate session data (combining the sessions)
    complete_exploration_df = complete_exploration_df.append(complete_hitpoints_df)
        
    # --------- Saving the Exploration ---------
    
    # If you want to save the exploration file, set 'Exploration_save_bool' to True
    if Exploration_save_bool == True:
        # saving the complete exploration
        try:
            complete_exploration_df.to_csv(PROCESSED_DATA_PATH + str(subject) + "_" + condition + "_CompleteHitpoints.csv")
            print("\t" + str(subject) + " " + condition + " eyetracking data saved")
        except:
            print("\tCould not save subject " + str(subject) + " " + condition + " eyetracking data!")
    

print('Done')

Subject 1023 started - 1/1
	ET: 1 normalized
	ET: 2 normalized
	ET: 3 normalized
	ET: 4 normalized
	ET: 5 normalized
	ET: 6 normalized
	ET: 7 normalized
	ET: 8 normalized
	ET: 9 normalized
	ET: 10 normalized
	1023 Single eyetracking data saved
Done


In [72]:
Result_df = pd.read_csv(PROCESSED_DATA_PATH+'1023_Single_CompleteHitpoints.csv')

In [73]:
Result_df

,Unnamed: 0,PathNumber,timeStampDataPointStart,timeStampDataPointEnd,hitObjectColliderName,ordinalOfHit,BitMask,hitPointOnObject.x,hitPointOnObject.y,hitPointOnObject.z,hitObjectColliderBoundsCenter.x,hitObjectColliderBoundsCenter.y,hitObjectColliderBoundsCenter.z,hmdPosition.x,hmdPosition.y,hmdPosition.z
0,0,0.0,0.001984,0.007936,StartZoneA0,1.0,3.0,251.315308,-0.877272,170.721466,251.851456,-2.287690,171.080597,250.958450,-0.494329,170.39097595214844
1,1,0.0,0.001984,0.007936,road_base_network.004,2.0,3.0,252.663849,-2.399850,172.152802,39.257874,-4.408301,51.768860,NaN,NaN,NaN
2,2,0.0,0.008432,0.008432,StartZoneA0,1.0,3.0,251.315308,-0.877272,170.721466,251.851456,-2.287690,171.080597,250.958450,-0.494329,170.39097595214844
3,3,0.0,0.008432,0.008432,road_base_network.004,2.0,3.0,252.663849,-2.399850,172.152802,39.257874,-4.408301,51.768860,NaN,NaN,NaN
4,4,0.0,0.011904,0.014880,road_base_network.004,1.0,3.0,252.691284,-2.400710,172.184113,39.257874,-4.408301,51.768860,250.988846,-0.494738,170.42933654785156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248447,18489,9.0,90.206415,90.206415,terrain_C.001,2.0,3.0,-282.912537,-0.626187,-180.557816,-491.456299,18.698456,-597.844604,NaN,NaN,NaN
248448,18490,9.0,90.225760,90.228735,pavement_C.002,1.0,3.0,-282.881683,-0.488985,-180.404495,-217.756897,-2.905127,-288.595245,-282.382690,1.367091,-177.7785186767578
248449,18491,9.0,90.225760,90.228735,terrain_C.001,2.0,3.0,-282.919189,-0.627347,-180.598740,-491.456299,18.698456,-597.844604,NaN,NaN,NaN
248450,18492,9.0,90.228735,90.228735,pavement_C.002,1.0,3.0,-282.881683,-0.488985,-180.404495,-217.756897,-2.905127,-288.595245,-282.382690,1.367091,-177.7785186767578


In [19]:
print("data points excluded for 1023: ");
(Result_df['PathNumber'] > 20).value_counts()


data points excluded for 1023: 


False    248452
Name: PathNumber, dtype: int64

# Performance Analysis

In [45]:
# Columns for consolidated data from all experiment files
pA_col_names = ['SubjectID', 'Condition', 'Dyadic?', 'Leader?', 'SessionID'];
for i in range(1,10):
    path_prefix = "P"+str(i);
    
    pA_col_names.append(path_prefix + ":Success")
    pA_col_names.append(path_prefix + ":TimeSec")
    pA_col_names.append(path_prefix + ":Time After Guide")
    pA_col_names.append(path_prefix + ":Distance Covered")

pA_col_names.append("#Success");


In [6]:
# Function to take subject ID and get distance walked for each path from subject's eyetracking files
def get_covered_distances(subID, condition):
    # TODO: have to think about no-hits for this?
#     return [0] * 10;
    
    # Get the consolidated hitpoints file which has HMD position data for each hitpoint
    # If hitpoint file not found, return 0 distance for all paths
    PROCESSED_DATA_PATH = './Results/' + condition + '/'
    try:
        filename = PROCESSED_DATA_PATH + str(subID) + '_' + condition +"_CompleteHitpoints.csv";
        data = pd.read_csv(filename);
    except:
        print('Could not read ' + filename);
        return[0] * 10;

    # Column names for intermediate DF and final DF respectively
    condense_col_names = ['PathNumber', 'xsqDiff', 'zsqDiff', 'eucDist']
    covered_dist_col_names = ['eucDist'];
    
    # Remove all second ordinal hits (HMD position is NA in consolidated hitpoint file)
    data = data.dropna(subset=['hmdPosition.x','hmdPosition.y','hmdPosition.z']);
    data = data.reset_index(drop=True)
    data['hmdPosition.z'] = data['hmdPosition.z'].astype(float);
    
    # Create intermediate and Final DF respectively
    condense_df = pd.DataFrame(index=range(len(data)), columns=condense_col_names)
    cov_dist_df = pd.DataFrame(index=range(10), columns=covered_dist_col_names);
    
    # Transfer over path number as is
    condense_df.PathNumber = data.PathNumber
    
    # Get the difference in the X- and Z-coordinates between each consecutive point 
    # Y-coordinate corresponds to HMD height and is ignored for map-distance covered
    condense_df.xsqDiff = data['hmdPosition.x'] - data['hmdPosition.x'].shift();
    condense_df.zsqDiff = data['hmdPosition.z'] - data['hmdPosition.z'].shift();
    
    # Square the x- and z- differences between each consecutive point
    condense_df.xsqDiff = condense_df.xsqDiff ** 2;
    condense_df.zsqDiff = condense_df.zsqDiff ** 2;
    
    # add the squared distances and take square root to get euclidean distance
    #    between each consecutive point
    condense_df.eucDist = (condense_df.xsqDiff + condense_df.zsqDiff) ** 0.5;
    
    # sum up the total euclidean distance grouped by path number to get distance 
    #    covered for each path
    cov_dist_df['eucDist'] = condense_df.groupby(['PathNumber'])['eucDist'].sum()
    
    # Return distances as list
    return list(cov_dist_df.eucDist)

get_covered_distances('1023', 'Single')


[20.246065926035442,
 1319.0761375256886,
 300.24722235596255,
 172.22456297365136,
 1919.323987975983,
 346.1267965206106,
 520.6331339390895,
 214.32876109412908,
 489.0953421457808,
 442.62558895940356]

In [123]:
# Create analysis data frame
perf_anal_df = pd.DataFrame(columns=pA_col_names);

# Loop through all subjects
for subject in subIDs:
    
    # Get distance covered per path for given subject
    covered_distances = get_covered_distances(subject, condition);
    
    # Read the experiment data for the subject
    sub_expt_data = {}
    sub_expt_file_name = PROCESSED_DATA_PATH + str(subject) + "_"+condition+"Pathfinding_Final.json";
    try:
        with open(sub_expt_file_name, 'r') as fp:
            sub_expt_data = json.load(fp);
    except:
        raise Exception("Could not read experiment file for subject " + str(subject));
    
    # Create the initial dictionary to store the experiment performance information for current subject 
    sub_per_data = {'SubjectID' : sub_expt_data['ParticipantID'],
                    'Condition' : 'B' if sub_expt_data['PathsReversed'] else 'A',
                    'Dyadic?' : sub_expt_data['IsDyadic'],
                    'Leader?' : sub_expt_data['IsLeader']
                   };
    
    # Store randomly generated (see cell somewher above) session ID if dyadic
    if sub_per_data['Dyadic?']: 
        sub_per_data['SessionID'] = dyadic_session_ids[sub_per_data['SubjectID']];
    else:
        sub_per_data['SessionID'] = 'NaN';
    
    # Get the path time limits for the current condition
    path_time_limits = path_time_limits_B if sub_expt_data['PathsReversed'] else path_time_limits_A;
    num_success = 0;
    
    # for each path (ignoring the tutorial path 0)
    for i in range(1,10):
        # Get the trial data from the experiment file for the current path
        current_trial_data = sub_expt_data['TrialData'][i];
        
        # Get the time for the path by subtracting start time from end time
        sub_per_data["P"+str(i)+":TimeSec"] = current_trial_data['EndMachineTimeStamp'] \
                                                - current_trial_data['StartMachineTimeStamp'];
        
        # Path was a success for subject if they did not use the path guide
        sub_per_data["P"+str(i)+":Success"] = not current_trial_data['GuideUsed'];
        
        # Count number of successes
        if sub_per_data["P"+str(i)+":Success"]: 
            num_success += 1;
        
        # Get the time after the guide appeared
        sub_per_data["P"+str(i)+":Time After Guide"] = sub_per_data["P"+str(i)+":TimeSec"] - path_time_limits[i-1];
        
        # Store the distance covered for the path
        sub_per_data["P"+str(i)+":Distance Covered"] = covered_distances[i];
    
    # Store the number of successes
    sub_per_data['#Success'] = num_success;
    
    # Normalize collected data and append to dataframe
    sub_df = pd.json_normalize(sub_per_data);
    perf_anal_df = perf_anal_df.append(sub_df);

print('Analysis Complete');

Could not read ./Results/Single/1004_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1005_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1008_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1010_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1011_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1013_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1017_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1018_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1019_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1021_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1022_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1054_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1055_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1056_Single_CompleteHitpoints.csv
Could not read ./Results/Single/1057_Single_CompleteHitpoints.csv
Could not 

In [122]:
# perf_anal_df

In [50]:
# Save to Single
perf_anal_df.to_csv(PROCESSED_DATA_PATH + 'Single_Performance_Analysis.csv', index=False);


In [55]:
# Save to Dyadic
perf_anal_df.to_csv(PROCESSED_DATA_PATH + 'Dyadic_Performance_Analysis.csv', index=False);


In [60]:
# Save to SingleC
perf_anal_df.to_csv(PROCESSED_DATA_PATH + 'SingleC_Performance_Analysis.csv', index=False);


In [118]:
# Display Collected Single Experiment Data
anal = pd.read_csv('./Results/Single/Single_Performance_Analysis.csv');
anal

,SubjectID,Condition,Dyadic?,Leader?,SessionID,P1:Success,P1:TimeSec,P1:Time After Guide,P1:Distance Covered,P2:Success,...,P7:Distance Covered,P8:Success,P8:TimeSec,P8:Time After Guide,P8:Distance Covered,P9:Success,P9:TimeSec,P9:Time After Guide,P9:Distance Covered,#Success
0,1004,B,False,False,NaN,False,449.037666,117.037666,0.000000,False,...,0.000000,False,390.436826,85.436826,0.000000,True,113.507971,-186.492029,0.000000,3
1,1005,A,False,False,NaN,True,158.458649,-141.541351,0.000000,True,...,0.000000,True,182.991541,-200.008459,0.000000,False,485.039520,153.039520,0.000000,6
2,1008,B,False,False,NaN,True,89.310862,-242.689138,0.000000,True,...,0.000000,True,90.587071,-214.412929,0.000000,True,106.655506,-193.344494,0.000000,8
3,1010,B,False,False,NaN,False,602.111310,270.111310,0.000000,True,...,0.000000,True,162.011336,-142.988664,0.000000,True,196.642525,-103.357475,0.000000,7
4,1011,B,False,False,NaN,True,102.925068,-229.074932,0.000000,True,...,0.000000,False,368.540978,63.540978,0.000000,True,217.615655,-82.384345,0.000000,7
5,1013,A,False,False,NaN,True,125.361833,-174.638167,0.000000,True,...,0.000000,True,136.801561,-246.198439,0.000000,False,438.497586,106.497586,0.000000,8
6,1017,B,False,False,NaN,True,261.689323,-70.310677,0.000000,False,...,0.000000,False,432.588424,127.588424,0.000000,True,289.393871,-10.606129,0.000000,4
7,1018,B,False,False,NaN,True,165.254440,-166.745560,0.000000,True,...,0.000000,False,407.066795,102.066795,0.000000,True,273.094684,-26.905316,0.000000,7
8,1019,A,False,False,NaN,True,100.749390,-199.250610,0.000000,True,...,0.000000,True,114.624974,-268.375026,0.000000,False,470.427692,138.427692,0.000000,7
9,1021,A,False,False,NaN,False,398.568231,98.568231,0.000000,True,...,0.000000,True,118.187025,-264.812975,0.000000,False,447.441651,115.441651,0.000000,6


In [119]:
# Display Collected Dyadic Experiment Data
anal = pd.read_csv('./Results/Dyadic/Dyadic_Performance_Analysis.csv');
anal

,SubjectID,Condition,Dyadic?,Leader?,SessionID,P1:Success,P1:TimeSec,P1:Time After Guide,P1:Distance Covered,P2:Success,...,P7:Distance Covered,P8:Success,P8:TimeSec,P8:Time After Guide,P8:Distance Covered,P9:Success,P9:TimeSec,P9:Time After Guide,P9:Distance Covered,#Success
0,1004,A,True,False,5575,True,187.532198,-112.467802,0,True,...,0,True,111.899616,-271.100384,0,True,119.377810,-212.622190,0,9
1,1005,B,True,True,5361,True,159.901736,-172.098264,0,True,...,0,True,105.293208,-199.706792,0,True,173.905779,-126.094221,0,8
2,1008,A,True,True,4957,True,117.755503,-182.244497,0,True,...,0,True,117.429136,-265.570864,0,True,258.906364,-73.093636,0,8
3,1011,A,True,True,1300,True,134.858309,-165.141691,0,True,...,0,True,119.176291,-263.823709,0,True,290.754437,-41.245563,0,7
4,1013,B,True,False,5062,True,99.181707,-232.818293,0,True,...,0,True,104.240194,-200.759806,0,True,97.303046,-202.696954,0,9
5,1017,A,True,False,1300,True,136.811322,-163.188678,0,True,...,0,True,122.440699,-260.559301,0,True,292.018808,-39.981192,0,9
6,1018,A,True,True,5128,True,263.787934,-36.212066,0,True,...,0,True,332.989794,-50.010206,0,True,111.840953,-220.159047,0,9
7,1021,B,True,True,7274,True,101.885863,-230.114137,0,True,...,0,True,62.674078,-242.325922,0,True,100.308087,-199.691913,0,9
8,1022,B,True,False,7274,True,103.525226,-228.474774,0,True,...,0,True,63.827329,-241.172671,0,True,101.237176,-198.762824,0,9
9,1023,B,True,True,5062,True,96.119958,-235.880042,0,True,...,0,True,103.161679,-201.838321,0,True,95.697365,-204.302635,0,9


In [120]:
# Display Collected Single Control Experiment Data
anal = pd.read_csv('./Results/SingleC/SingleC_Performance_Analysis.csv');
anal

,SubjectID,Condition,Dyadic?,Leader?,SessionID,P1:Success,P1:TimeSec,P1:Time After Guide,P1:Distance Covered,P2:Success,...,P7:Distance Covered,P8:Success,P8:TimeSec,P8:Time After Guide,P8:Distance Covered,P9:Success,P9:TimeSec,P9:Time After Guide,P9:Distance Covered,#Success
0,1010,A,False,False,NaN,True,103.283452,-196.716548,0,True,...,0,True,149.935175,-233.064825,0,False,349.954884,17.954884,0,7
1,1019,B,False,False,NaN,True,121.585427,-210.414573,0,True,...,0,True,62.756872,-242.243128,0,True,126.389681,-173.610319,0,9
2,1056,B,False,False,NaN,True,99.185621,-232.814379,0,True,...,0,False,376.036466,71.036466,0,True,101.787637,-198.212363,0,7
3,1068,A,False,False,NaN,True,129.268349,-170.731651,0,True,...,0,True,117.311292,-265.688708,0,False,474.197720,142.197720,0,7
4,1072,A,False,False,NaN,True,106.815908,-193.184092,0,True,...,0,True,113.280268,-269.719732,0,True,150.568494,-181.431506,0,8
5,1073,A,False,False,NaN,False,352.014228,52.014228,0,True,...,0,True,175.417123,-207.582877,0,False,454.078002,122.078002,0,5
6,1075,B,False,False,NaN,True,96.997154,-235.002846,0,True,...,0,True,58.880095,-246.119905,0,True,97.315090,-202.684910,0,9
7,1077,A,False,False,NaN,True,102.892725,-197.107275,0,True,...,0,True,211.720585,-171.279415,0,True,100.524325,-231.475675,0,7
8,1079,B,False,False,NaN,True,294.374525,-37.625475,0,True,...,0,False,345.073664,40.073664,0,True,274.738379,-25.261621,0,7
9,1080,A,False,False,NaN,True,137.255427,-162.744573,0,True,...,0,True,125.162962,-257.837038,0,False,432.332893,100.332893,0,6


### Single Pathfinding Descriptive Statistics

In [111]:

# Per Condition, Per Path: Average Time, SD Time, Average Success, Average Time after Fail
# Per Condition Success Average

# Columns for the descriptive statistics that we want to consider
path_anal_cols = ['PathNum', '+ Rate', 'Limit', 'Avg. T (+)', 'SD T (+)', 'Avg. T (after -)', \
                  'SD T (after -)', 'Avg D (+)', 'SD D (+)']

# Read single pathfinding performance data
anal = pd.read_csv('./Results/Single/Single_Performance_Analysis.csv');

# Separate into Conditions A and B
A_anal = anal[anal.Condition == 'A']
B_anal = anal[anal.Condition == 'B']

# Get average success rate and std across participants for both conditions
A_succ_avg = A_anal['#Success'].mean() 
A_anal_succ_std  = A_anal['#Success'].std()
B_succ_avg = B_anal['#Success'].mean()
B_anal_succ_std  = B_anal['#Success'].std()


# Starting extraction of statistics for condition A

# Filter information from the dataframe that pertains to path performance (1-9)
A_paths = A_anal.filter(regex="P.*")
A_path_stats = pd.DataFrame(columns = path_anal_cols);

curr_row = {};
successed = [];
fails = []

# Iterate through all the columns containing path information
# Each path has 4 columns
for idx, column in enumerate(A_paths):
    coltype = idx % 4;
    # Store the path number and the corresponding time limit
    curr_row['PathNum'] = int(idx / 4) + 1;
    curr_row['Limit'] = path_time_limits_A[curr_row['PathNum'] - 1]
    
    # First column describes path success or not. Calculate success rate.
    if coltype == 0:
        successes = A_paths[column];
        fails = ~successes;
        curr_row['+ Rate'] = successes.sum() / successes.count();
        
    # Second column describes time taken. Compute average and SD time for all succesful paths
    elif coltype == 1:
        curr_row['Avg. T (+)'] = A_paths[column][successes].mean();
        curr_row['SD T (+)'] = A_paths[column][successes].std();
        
    # Third column describes time taken after path time limit. Compute avg and SD for all failed paths
    elif coltype == 2:
        curr_row['Avg. T (after -)'] = A_paths[column][fails].mean();
        curr_row['SD T (after -)'] = A_paths[column][fails].std();
    
    # Fourth column describes distance covered. Compute avg and SD for all successes
    elif coltype == 3:
        curr_row['Avg D (+)'] = A_paths[column][successes].mean();
        curr_row['SD D (+)'] = A_paths[column][successes].std();
        
        A_path_stats = A_path_stats.append(curr_row, ignore_index=True)
        curr_row = {}

# Save to CSV
A_path_stats.to_csv(PROCESSED_DATA_PATH + 'A_Path_Stats.csv');


# Starting extraction of descriptive statistics for paths in condition B
# Procedure is exactly the same as above, refer to above comments. 

B_paths = B_anal.filter(regex="P.*")
B_path_stats = pd.DataFrame(columns = path_anal_cols);

curr_row = {};
successed = [];
fails = []

for idx, column in enumerate(B_paths):
    coltype = idx % 4;
    curr_row['PathNum'] = int(idx / 4) + 1;
    curr_row['Limit'] = path_time_limits_B[curr_row['PathNum'] - 1]
    if coltype == 0:
        successes = B_paths[column];
        fails = ~successes;
        curr_row['+ Rate'] = successes.sum() / successes.count();
    elif coltype == 1:
        curr_row['Avg. T (+)'] = B_paths[column][successes].mean();
        curr_row['SD T (+)'] = B_paths[column][successes].std();
    elif coltype == 2:
        curr_row['Avg. T (after -)'] = B_paths[column][fails].mean();
        curr_row['SD T (after -)'] = B_paths[column][fails].std();
    elif coltype == 3:
        curr_row['Avg D (+)'] = B_paths[column][successes].mean();
        curr_row['SD D (+)'] = B_paths[column][successes].std();
        
        B_path_stats = B_path_stats.append(curr_row, ignore_index=True)
        curr_row = {}
    
B_path_stats.to_csv(PROCESSED_DATA_PATH + 'B_Path_Stats.csv');

In [128]:
print('Avg Successes for Condition A: ' + str(A_succ_avg));
print('SD Successes for Condition A: ' + str(A_anal_succ_std));

# Descriptive statistics per path for condition A
A_path_stats

Avg Successes for Condition A: 5.666666666666667
SD Successes for Condition A: 2.0150945537631877


,PathNum,+ Rate,Limit,Avg. T (+),SD T (+),Avg. T (after -),SD T (after -),Avg D (+),SD D (+)
0,1.0,0.583333,300.0,161.914439,54.236144,100.005741,38.805747,188.439448,498.563917
1,2.0,0.916667,305.0,103.495294,41.733701,60.849056,NaN,27.295202,90.527944
2,3.0,0.833333,234.0,52.072322,35.897732,93.972566,103.429060,17.222456,54.462189
3,4.0,0.166667,332.0,196.542249,58.004837,120.673807,41.780004,0.000000,0.000000
4,5.0,0.500000,268.0,79.653422,9.573732,96.395639,65.235597,57.687799,141.305673
5,6.0,0.833333,394.0,155.129403,71.633193,62.425426,9.019406,52.063313,164.638653
6,7.0,0.750000,152.0,57.555982,15.655423,150.006501,98.169669,23.814307,71.442920
7,8.0,0.833333,383.0,159.703784,63.684079,48.994737,56.295677,48.909534,154.665527
8,9.0,0.250000,332.0,98.339506,17.564382,132.522930,77.269232,147.541863,255.550003


In [130]:
print('Avg Successes for Condition B: ' + str(B_succ_avg));
print('SD Successes for Condition B: ' + str(B_anal_succ_std));

B_path_stats

Avg Successes for Condition B: 6.214285714285714
SD Successes for Condition B: 1.5281246137553164


,PathNum,+ Rate,Limit,Avg. T (+),SD T (+),Avg. T (after -),SD T (after -),Avg D (+),SD D (+)
0,1.0,0.714286,332.0,146.107746,62.410763,153.370460,89.679451,0.0,0.0
1,2.0,0.642857,383.0,161.914168,85.031437,123.339809,46.854770,0.0,0.0
2,3.0,0.571429,152.0,49.831135,3.748735,129.493369,57.549610,0.0,0.0
3,4.0,0.928571,394.0,206.320719,97.617372,79.476636,NaN,0.0,0.0
4,5.0,0.000000,268.0,NaN,NaN,102.886259,74.888210,NaN,NaN
5,6.0,0.714286,332.0,167.674285,58.569429,95.827939,12.060102,0.0,0.0
6,7.0,1.000000,234.0,62.683658,51.683104,NaN,NaN,0.0,0.0
7,8.0,0.642857,305.0,177.997099,78.367980,98.827377,25.200365,0.0,0.0
8,9.0,1.000000,300.0,150.240288,67.216606,NaN,NaN,0.0,0.0


In [121]:
# Averages for each category and standard deviation
# Look for outliers
# Average speed? - Distance over time - Compare to final performance
#    for individual paths and for participants overall

# TODO - think about the relevance of individual analysis and path analysis
# TODO - time before guide appeared for success - individual subject and for each path?
# TODO - same performance for dyadic, stare at statistics and generate some observations